In [19]:
import requests
from datetime import datetime
from typing import Dict, Optional, Union
import os
from langchain_core.tools import tool

In [21]:
OPENWEATHER_API_KEY = ''

In [33]:
class WeatherTool:
    """A tool for retrieving weather information using OpenWeatherMap API."""
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Initialize the WeatherTool.
        
        Args:
            api_key: OpenWeatherMap API key. If not provided, will try to get it from OPENWEATHER_API_KEY env variable.
        """
        self.api_key = api_key or OPENWEATHER_API_KEY
        
        if not self.api_key:
            raise ValueError(
                "OpenWeatherMap API key is required. Set the OPENWEATHER_API_KEY environment variable "
                "or pass it directly to the WeatherTool constructor."
            )
        
        self.base_url = "https://api.openweathermap.org/data/2.5"
        self.geocoding_url = "http://api.openweathermap.org/geo/1.0/direct"
    
    def geocode_address(self, address: str) -> Dict:
        """
        Convert a street address to latitude and longitude using OpenWeatherMap Geocoding API.
        
        Args:
            address: Full address string (e.g., '75 Saint Alphonsus Street, Boston, MA')
            
        Returns:
            Dict containing latitude and longitude
        """
        params = {
            "q": address,
            "limit": 1,
            "appid": self.api_key
        }
        
        response = requests.get(self.geocoding_url, params=params)
        
        if response.status_code != 200:
            error_msg = response.json().get("message", "Unknown error")
            raise Exception(f"Error geocoding address: {error_msg} (Code: {response.status_code})")
        
        results = response.json()
        
        if not results:
            raise Exception(f"No geocoding results found for address: {address}")
        
        location = results[0]
        return {
            "lat": location.get("lat"),
            "lon": location.get("lon"),
            "name": location.get("name"),
            "state": location.get("state", ""),
            "country": location.get("country", "")
        }
    
    def get_forecast_by_coords(self, lat: float, lon: float, units: str = "metric") -> Dict:
        """
        Get weather forecast by latitude and longitude.
        
        Args:
            lat: Latitude
            lon: Longitude
            units: Units of measurement. Options: 'standard', 'metric', or 'imperial'
            
        Returns:
            Dict containing forecast information
        """
        endpoint = f"{self.base_url}/forecast"
        params = {
            "lat": lat,
            "lon": lon,
            "appid": self.api_key,
            "units": units,
            "cnt": 40  # Maximum number of timestamps (5 days with 3-hour steps)
        }
        
        response = requests.get(endpoint, params=params)
        
        if response.status_code != 200:
            error_msg = response.json().get("message", "Unknown error")
            raise Exception(f"Error fetching forecast: {error_msg} (Code: {response.status_code})")
        
        return response.json()
    
    def format_forecast_for_date(self, forecast_data: Dict, date_str: str, units: str = "metric") -> str:
        """
        Format forecast data for a specific date.
        
        Args:
            forecast_data: Forecast data from get_forecast_by_coords
            date_str: Date string in format 'YYYY-MM-DD'
            units: Units used in the data ('metric', 'imperial', 'standard')
            
        Returns:
            Formatted forecast information for the specified date
        """
        temp_unit = "°C" if units == "metric" else "°F" if units == "imperial" else "K"
        city_name = forecast_data.get("city", {}).get("name", "Unknown location")
        city_country = forecast_data.get("city", {}).get("country", "")
        
        try:
            target_date = datetime.strptime(date_str, "%Y-%m-%d").date()
            
            # Filter forecasts for the target date
            forecasts = forecast_data.get("list", [])
            filtered_forecasts = [
                f for f in forecasts 
                if datetime.fromtimestamp(f.get("dt")).date() == target_date
            ]
            
            if not filtered_forecasts:
                return f"No forecast available for {date_str} at the specified location."
            
            # Format the response
            location_name = f"{city_name}, {city_country}" if city_country else city_name
            result = f"Weather forecast for {location_name} on {date_str}:\n\n"
            
            # Add each time period forecast
            for f in filtered_forecasts:
                dt = datetime.fromtimestamp(f.get("dt"))
                time_str = dt.strftime("%H:%M")
                
                temp = f.get("main", {}).get("temp", "N/A")
                feels_like = f.get("main", {}).get("feels_like", "N/A")
                humidity = f.get("main", {}).get("humidity", "N/A")
                description = f.get("weather", [{}])[0].get("description", "N/A")
                wind_speed = f.get("wind", {}).get("speed", "N/A")
                
                pop = f.get("pop", 0) * 100  # Probability of precipitation (as percentage)
                
                result += f"{time_str} - {description.capitalize()}\n"
                result += f"  Temperature: {temp}{temp_unit}, feels like {feels_like}{temp_unit}\n"
                result += f"  Humidity: {humidity}%, Wind: {wind_speed} m/s\n"
                result += f"  Chance of precipitation: {pop:.0f}%\n\n"
            
            # Add a summary
            avg_temp = sum(f.get("main", {}).get("temp", 0) for f in filtered_forecasts) / len(filtered_forecasts)
            avg_pop = sum(f.get("pop", 0) for f in filtered_forecasts) / len(filtered_forecasts) * 100
            
            # Count weather conditions
            conditions = {}
            for f in filtered_forecasts:
                condition = f.get("weather", [{}])[0].get("description", "N/A")
                conditions[condition] = conditions.get(condition, 0) + 1
            
            # Get the most common condition
            most_common_condition = max(conditions.items(), key=lambda x: x[1])[0]
            
            result += f"SUMMARY: {most_common_condition.capitalize()} for most of the day. "
            result += f"Average temperature: {avg_temp:.1f}{temp_unit}. "
            result += f"Chance of precipitation: {avg_pop:.0f}%."
            
            return result
            
        except ValueError:
            return f"Invalid date format. Please use YYYY-MM-DD."


def check_weather_for_address(address: str, date: str, units: str = "metric") -> str:
    """
    Check weather forecast for a specific address and date.
    
    Args:
        address: Full street address (e.g., '75 Saint Alphonsus Street, Boston, MA')
        date: Date in format 'YYYY-MM-DD'
        units: Units of measurement ('metric' for Celsius, 'imperial' for Fahrenheit)
        
    Returns:
        String with formatted weather forecast for the specified date
    """
    try:
        weather_tool = WeatherTool()
        
        # Convert address to coordinates
        location = weather_tool.geocode_address(address)
        
        # Get forecast data
        forecast_data = weather_tool.get_forecast_by_coords(
            location["lat"], 
            location["lon"], 
            units=units
        )
        
        # Format forecast for the specific date
        return weather_tool.format_forecast_for_date(forecast_data, date, units)
        
    except Exception as e:
        return f"Error retrieving weather information: {str(e)}"


if __name__ == "__main__":
    specific_address = "Boston"
    specific_date = "2025-04-19"  # April 19, 2025
    
    print(f"FORECAST FOR {specific_address} ON {specific_date}:")
    print(check_weather_for_address(specific_address, specific_date))
    
    # Using imperial units (Fahrenheit)
    print("\nFORECAST WITH IMPERIAL UNITS:")
    print(check_weather_for_address(specific_address, specific_date, units="imperial"))

FORECAST FOR Boston ON 2025-04-19:
Weather forecast for Boston, US on 2025-04-19:

02:00 - Overcast clouds
  Temperature: 13.05°C, feels like 12.24°C
  Humidity: 70%, Wind: 6.47 m/s
  Chance of precipitation: 0%

05:00 - Overcast clouds
  Temperature: 12.26°C, feels like 11.58°C
  Humidity: 78%, Wind: 5.75 m/s
  Chance of precipitation: 0%

08:00 - Overcast clouds
  Temperature: 14.07°C, feels like 13.52°C
  Humidity: 76%, Wind: 5.54 m/s
  Chance of precipitation: 0%

11:00 - Overcast clouds
  Temperature: 20.56°C, feels like 20.16°C
  Humidity: 57%, Wind: 6.73 m/s
  Chance of precipitation: 0%

14:00 - Overcast clouds
  Temperature: 25.42°C, feels like 25.22°C
  Humidity: 46%, Wind: 6.07 m/s
  Chance of precipitation: 0%

17:00 - Overcast clouds
  Temperature: 24.68°C, feels like 24.51°C
  Humidity: 50%, Wind: 5.11 m/s
  Chance of precipitation: 0%

20:00 - Light rain
  Temperature: 21.74°C, feels like 21.64°C
  Humidity: 64%, Wind: 4.07 m/s
  Chance of precipitation: 25%

23:00 - Ove